# BikeDNA
https://github.com/anerv/BikeDNA

## (1a) Load & Process OSM data

This notebook:
- Loads the polygon defining the study area and then creates a grid overlay for the study area.
- Downloads street network data for the study area using OSMnx.
- Creates a network only with bicycle infrastructure (with queries defined in `config.yml`).
- Creates additional attributes in the data to be used in the analysis.


<div class="alert alert-block alert-info">
<b>Prerequisites &amp; Input/Output</b>
<p>
<span style="font-family:courier;">config.yml</span> must be set in advance.
    
Output files of this notebook are saved to the <span style="font-family:courier;">../data/OSM/'study_area'/processed/</span> subfolder and used in notebooks 2a, 3a and 3b.

In the process of creating a graph from the OSM data, each edge is assigned a unique edge ID (<i>not</i> OSM ID). To relate the edge ID back to the OSM ID, the <span style="font-family:courier;">osm_edges_simplified</span> dataframe can be used.
</p>
</div>

**Sections**

1. Load data for study area & define grid used for computation local results

2. Load OSM data

***

### 1. Load data for study area & define grid

This step:

- Loads settings for the analysis from the config file.
- Reads data for the study area.
- Creates a grid overlay of the study area, with grid cell size as defined in configuration file `config.yml`.

<div class="alert alert-block alert-warning">
<b>Troubleshooting</b>
<p>
If in the cell below the functions located in the <i>src</i> folder do not load properly: Check that <span style="font-family:courier;">pip install -e .</span> was run successfully (see instructions in <span style="font-family:courier;">README.md</span>).
</p>
</div>

In [ ]:
# Load libraries, settings and data

import json
import pickle
import warnings
from datetime import datetime

import contextily as cx
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import numpy as np
import yaml

from src import evaluation_functions as eval_func
from src import graph_functions as graph_func
from src import simplification_functions as simp_func
from src import plotting_functions as plot_func

%run ../settings/plotting.py
%run ../settings/yaml_variables.py
%run ../settings/paths.py

warnings.filterwarnings("ignore")

#### Read in data for study area

The study area is defined by the user-provided polygon. It will be used for the computation of **global** results, i.e. for the entire study area.

In [ ]:
# Read polygon for study area

study_area_poly = gpd.read_file(study_area_poly_fp)

if study_area_poly.crs == None:
    print("Please assign a crs to the study area polygon!")

if study_area_poly.crs != study_crs:
    study_area_poly = study_area_poly.to_crs(study_crs)

assert study_area_poly.crs == study_crs

area = study_area_poly.area.values[0]
print(f"The size of the study area is {area / 1000000:.2f} square kilometers.")
print("This study area will be used for global (study area level) analysis:")

fig, ax = plt.subplots(1, figsize=pdict["fsmap"])

study_area_poly.plot(ax=ax, edgecolor=pdict["base"], facecolor='none',alpha=pdict["alpha_back"],hatch='/')

ax.set_axis_off()

cx.add_basemap(ax, crs=study_area_poly.crs, source=cx_tile_1)

ax.set_title(f"Study area: {study_area} ({area / 1000000:.2f} sqkm)")

fig.savefig(osm_results_static_maps_fp + "area_size_OSM.png", dpi=pdict["dpi"])

#### Create analysis grid

<div class="alert alert-block alert-info">
<b>User configurations</b>
<p>
The grid is used to compute <b>local</b> results for many of the quality metrics, i.e. on grid cell level. The cell size of the grid cells must be specified in the configuration file. Smaller grid cells will increase the computation time but give a more granular result. The grid cell size should not be smaller than the scale at which you would expect actual differences in data quality.
</p>
</div>

In [ ]:
grid = eval_func.create_grid_geometry(study_area_poly, grid_cell_size)

grid["grid_id"] = grid.index

fig, ax = plt.subplots(1, figsize=pdict["fsmap"])

grid_cell_area = (grid_cell_size / 1000) ** 2  # in km2
print(
    f"The grid contains {len(grid)} square cells with a side length of {grid_cell_size}m and an area of {grid_cell_area:.2f} square kilometers."
)
print("This grid will be used for local (grid cell level) analysis:")

grid.plot(ax=ax, facecolor="none", edgecolor=pdict["base"], alpha=pdict["alpha_back"])

ax.set_axis_off()

cx.add_basemap(ax, crs=study_area_poly.crs, source=cx_tile_1)

ax.set_title(
    f"Study area: {study_area}, number of grid cells: {len(grid)}, grid cell side length: {grid_cell_size}m"
)

fig.savefig(osm_results_static_maps_fp + "area_grid_OSM.png", dpi=pdict["dpi"])

### 2. Load & process OSM data

This step:

- Downloads data from OpenStreetMap using OSMnx.
- Projects the data to the chosen CRS.
- Creates a subnetwork consisting only of bicycle infrastructure.
- Classifies all edges in the bicycle network based on whether they are protected or unprotected bicycle infrastructure, how they have been digitized, and whether they allow for bidirectional travel or not
- Simplifies the network *(to read more about the modified OSMnx simplification (Boeing, 2017) used here, we refer to this [GitHub repository](https://github.com/anerv/NERDS_osmnx) which contains both the simplification functions, explanation of the logic and a demonstration)*.
- Creates copies of all edge and node datasets indexed by their intersecting grid cell.

Depending on the size of your study area, getting data through OSMnx can take a little while (for a city around the size of Copenhagen, it usually takes 1-2 minutes).

<div class="alert alert-block alert-info">
<b>User configurations</b>
<p>
In the following step, the dedicated bicycle infrastructure is extracted from the general street network. Depending on the type of bicycle infrastructure and OSM tagging tradition in your area, the queries used for retrieving the bicycle network might need to be adapted (this is done in the configuration file under <i>bicycle_infrastructure_queries</i>).
</p>
</div>

In [ ]:
# Download OSM data

ox.settings.use_cache = False
ox.settings.useful_tags_way = osm_way_tags

graph_osm = ox.graph_from_polygon(
    study_area_poly.to_crs("EPSG:4326").loc[0, "geometry"],
    network_type="bike",
    simplify=False,
    retain_all=True,
    truncate_by_edge=False,
    clean_periphery=True,
)

print("OSM data retrieved successfully!")

In [ ]:
# Convert into graph

# Project graph to chosen crs
graph_osm = ox.project_graph(graph_osm, to_crs=study_crs)

# Get osm_edges and osm_nodes
osm_nodes, osm_edges = ox.graph_to_gdfs(graph_osm)

# Clean up column names (removing ':' and upper case letters)
osm_edges = graph_func.clean_col_names(osm_edges)
osm_nodes = graph_func.clean_col_names(osm_nodes)

# Add attribute on whether bicycle infra exist or not (to be used by e.g. simplification function)
osm_edges["bicycle_infrastructure"] = "no"

for q in bicycle_infrastructure_queries.values():

    try:
        ox_filtered = osm_edges.query(q)

    except Exception:
        print("Exception occured when quering with:", q)
        print("Please check if the columns used in the query are present in the data")

    osm_edges.loc[ox_filtered.index, "bicycle_infrastructure"] = "yes"

osm_edges.bicycle_infrastructure.value_counts()

bicycle_edges = osm_edges.loc[osm_edges.bicycle_infrastructure == "yes"].copy()

bicycle_graph = graph_osm.edge_subgraph(bicycle_edges.index)
print("Successfully created network only with bicycle infrastructure!")
print("\n")

assert len(bicycle_edges) == len(bicycle_graph.edges)

bicycle_nodes, bicycle_edges = ox.graph_to_gdfs(bicycle_graph)
bicycle_edges = graph_func.clean_col_names(bicycle_edges)
bicycle_nodes = graph_func.clean_col_names(bicycle_nodes)

bicycle_edges["bicycle_infrastructure"] = "yes"
bicycle_infrastructure_dict = bicycle_edges["bicycle_infrastructure"].to_dict()
nx.set_edge_attributes(
    bicycle_graph, bicycle_infrastructure_dict, "bicycle_infrastructure"
)

# Add "multiple edge" attribute to each edge of the graph
for e in bicycle_graph.edges:
    bicycle_graph.edges[e]["multiedge"] = 0

# Count multiple edges and store info at key 0
# (simplification routine keeps only key 0)
for u, v, k in bicycle_graph.edges:
    if k > 0:
        bicycle_graph.edges[u, v, 0]["multiedge"] += 1

# Define whether bicycle infrastructure is in both or one direction and whether it is one- or bidirectional
bicycle_edges = eval_func.simplify_bicycle_tags(bicycle_edges)

for key, value in bicycle_edges.bicycle_bidirectional.value_counts().items():
    perc = np.round(100*value/len(bicycle_edges),2)
    print(f"Number of edges where 'bicycle_bidirectional' is {key}: {value} out of {len(bicycle_edges)} ({perc}%)")
print("\n")

for key, value in bicycle_edges.bicycle_geometries.value_counts().items():
    perc = np.round(100*value/len(bicycle_edges),2)
    print(f"Number of edges where the geometry type is '{key}': {value} out of {len(bicycle_edges)} ({perc}%)")
print("\n")

bicycle_bidirectional_dict = bicycle_edges["bicycle_bidirectional"].to_dict()
nx.set_edge_attributes(
    bicycle_graph, bicycle_bidirectional_dict, "bicycle_bidirectional"
)

bicycle_geometries_dict = bicycle_edges["bicycle_geometries"].to_dict()
nx.set_edge_attributes(bicycle_graph, bicycle_geometries_dict, "bicycle_geometries")

# Classify edges as protected or unprotected
bicycle_edges = eval_func.define_protected_unprotected(
    bicycle_edges, osm_bicycle_infrastructure_type
)

# Set edges attributes for column protected
bicycle_protected_dict = bicycle_edges["protected"].to_dict()
nx.set_edge_attributes(bicycle_graph, bicycle_protected_dict, "protected")

for key, value in bicycle_edges.protected.value_counts().items():
    perc = np.round(100*value/len(bicycle_edges),2)
    print(f"Number of edges where the protection level is '{key}': {value} out of {len(bicycle_edges)} ({perc}%)")
print("\n")

# Simplify bicycle network
bicycle_graph_simplified = simp_func.simplify_graph(
    bicycle_graph,
    attributes=[
        "bicycle_infrastructure",
        "bicycle_bidirectional",
        "bicycle_geometries",
        "protected",
    ],
    remove_rings=False,
)

# Get undirected network representation
bicycle_graph_simplified = ox.get_undirected(bicycle_graph_simplified)
bicycle_graph = ox.get_undirected(bicycle_graph)

bicycle_nodes_simplified, bicycle_edges_simplified = ox.graph_to_gdfs(
    bicycle_graph_simplified, nodes=True
)

# Measure the length of bicycle infrastructure (!=from length of edges)
bicycle_edges_simplified["length"] = bicycle_edges_simplified.geometry.length
bicycle_edges_simplified["infrastructure_length"] = bicycle_edges_simplified.apply(
    lambda x: eval_func.measure_infrastructure_length(
        edge=x.geometry,
        geometry_type=x.bicycle_geometries,
        bidirectional=x.bicycle_bidirectional,
        bicycle_infrastructure=x.bicycle_infrastructure,
    ),
    axis=1,
)

# Creating columns with unique feature id (required by some functions)
bicycle_edges["edge_id"] = bicycle_edges.reset_index().index
bicycle_edges_simplified["edge_id"] = bicycle_edges_simplified.reset_index().index

osm_id_dict = bicycle_edges["edge_id"].to_dict()
nx.set_edge_attributes(bicycle_graph, osm_id_dict, "edge_id")

osm_id_dict_s = bicycle_edges_simplified["edge_id"].to_dict()
nx.set_edge_attributes(bicycle_graph_simplified, osm_id_dict_s, "edge_id")

# Saving data to graphs
osm_l_dict = osm_edges["length"].to_dict()
nx.set_edge_attributes(bicycle_graph, osm_l_dict, "length")

osm_l_dict_s = bicycle_edges_simplified["length"].to_dict()
nx.set_edge_attributes(bicycle_graph_simplified, osm_l_dict_s, "length")

osm_il_dict = bicycle_edges_simplified["infrastructure_length"].to_dict()
nx.set_edge_attributes(bicycle_graph_simplified, osm_il_dict, "infrastructure_length")

# Overview of data from OSM
graph_area = eval_func.get_graph_area(bicycle_nodes, study_area_poly, study_crs)
print(f"The graph covers an area of {graph_area/ 1000000:.2f} square kilometers")
print(
    f"The length of the OSM network with bicycle infrastructure is {bicycle_edges_simplified.infrastructure_length.sum()/1000 :.2f} kilometers"
)

In [ ]:
# Plot global statistics (by network length)

ymax = bicycle_edges_simplified.infrastructure_length.sum() * 1.2

# attribute: bidirectional
x = []
y = []

for value in np.unique(bicycle_edges.bicycle_bidirectional):
    x.append(str(value))
    y.append(bicycle_edges[bicycle_edges["bicycle_bidirectional"]==value].length.sum()/1000) # conversion m to km

plot_func.make_bar_plot(
    data = y,
    bar_labels = x,
    y_label = "Network edge length [km]",
    x_positions = [i for i in range(len(x))],
    title = f"{study_area}, OSM network: edges by bidirectionality",
    bar_colors = len(x)*[pdict["osm_base"]],
    filepath = osm_results_plots_fp + "edge_global_bidirectional.png",
    ylim = ymax
);

# attribute: bicycle geometry (mapping type)
x = []
y = []

for value in np.unique(bicycle_edges.bicycle_geometries):
    x.append(value)
    y.append(bicycle_edges[bicycle_edges["bicycle_geometries"]==value].length.sum())

plot_func.make_bar_plot(
    data = y,
    bar_labels = x,
    y_label = "Network edge length [km]",
    x_positions = [i for i in range(len(x))],
    title = f"{study_area}, OSM network: edges by mapping type",
    bar_colors = len(x)*[pdict["osm_base"]],
    filepath = osm_results_plots_fp + "edge_global_mappingtype.png",
    ylim = ymax
);

# attribute: protection
x = []
y = []

for value in np.unique(bicycle_edges.protected):
    x.append(value)
    y.append(bicycle_edges[bicycle_edges["protected"]==value].length.sum())

plot_func.make_bar_plot(
    data = y,
    bar_labels = x,
    y_label = "Network edge length [km]",
    x_positions = [i for i in range(len(x))],
    title = f"{study_area}, OSM network: edges by infrastructure type",
    bar_colors = len(x)*[pdict["osm_base"]],
    filepath = osm_results_plots_fp + "edge_global_infrastructuretype.png",
    ylim = ymax
);

In [ ]:
# Plot network

fig, ax = plt.subplots(1, figsize=pdict["fsmap"])

bicycle_edges_simplified.plot(ax=ax, color=pdict["osm_base"], linewidth=pdict["line_base"])

bicycle_nodes_simplified.plot(ax=ax, color=pdict["osm_emp"], markersize=pdict["mark_emp"])

study_area_poly.plot(ax=ax, edgecolor=pdict["base"], facecolor="None", linewidth=1)

cx.add_basemap(ax=ax, crs=study_area_poly.crs, source=cx_tile_2)

ax.set_axis_off()

ax.set_title(f"{study_area}, network created from OSM data")

fig.savefig(osm_results_static_maps_fp + "area_network_OSM.png", dpi=pdict["dpi"])

In [ ]:
# Joining grid cell id to network elements

osm_edges_simp_joined = gpd.overlay(
    bicycle_edges_simplified.reset_index(), grid, how="intersection"
)
osm_edges_joined = gpd.overlay(bicycle_edges.reset_index(), grid, how="intersection")

osm_nodes_simp_joined = gpd.overlay(
    bicycle_nodes_simplified.reset_index(), grid, how="intersection"
)
osm_nodes_joined = gpd.overlay(bicycle_nodes.reset_index(), grid, how="intersection")

# Count features in each grid cell
all_data_osm = [
    osm_edges_joined,
    osm_nodes_joined,
    osm_edges_simp_joined,
    osm_nodes_simp_joined,
]
labels_osm = ["osm_edges", "osm_nodes", "osm_simplified_edges", "osm_simplified_nodes"]

for data, label in zip(all_data_osm, labels_osm):

    df = eval_func.count_features_in_grid(data, label)

    grid = eval_func.merge_results(grid, df, "left")

# Recomputing infrastructure length for data joined to grid
osm_edges_simp_joined["length"] = osm_edges_simp_joined["geometry"].length
osm_edges_simp_joined["infrastructure_length"] = osm_edges_simp_joined.apply(
    lambda x: eval_func.measure_infrastructure_length(
        edge=x.geometry,
        geometry_type=x.bicycle_geometries,
        bidirectional=x.bicycle_bidirectional,
        bicycle_infrastructure=x.bicycle_infrastructure,
    ),
    axis=1,
)

assert round(bicycle_edges_simplified.infrastructure_length.sum() / 10000, 0) == round(
    osm_edges_simp_joined.infrastructure_length.sum() / 10000, 0
)

In [ ]:
# Save data

bicycle_nodes["osmid"] = bicycle_nodes.index
bicycle_nodes_simplified["osmid"] = bicycle_nodes_simplified.index
assert len(bicycle_nodes) == len(bicycle_nodes.osmid.unique())
assert len(bicycle_edges) == len(bicycle_edges.edge_id.unique())
assert len(bicycle_edges_simplified) == len(bicycle_edges_simplified.edge_id.unique())
assert len(bicycle_nodes_simplified) == len(bicycle_nodes_simplified.osmid.unique())

assert "infrastructure_length" in bicycle_edges_simplified.columns
assert "length" in bicycle_edges.columns

with open(osm_nodes_fp, "wb") as f:
    pickle.dump(bicycle_nodes, f)

with open(osm_edges_fp, "wb") as f:
    pickle.dump(bicycle_edges, f)

with open(osm_nodes_simplified_fp, "wb") as f:
    pickle.dump(bicycle_nodes_simplified, f)

with open(osm_edges_simplified_fp, "wb") as f:
    pickle.dump(bicycle_edges_simplified, f)

# Joined data
with open(osm_nodes_joined_fp, "wb") as f:
    pickle.dump(osm_nodes_joined, f)

with open(osm_edges_joined_fp, "wb") as f:
    pickle.dump(osm_edges_joined, f)

with open(osm_nodes_simplified_joined_fp, "wb") as f:
    pickle.dump(osm_nodes_simp_joined, f)

with open(osm_edges_simplified_joined_fp, "wb") as f:
    pickle.dump(osm_edges_simp_joined, f)

print("OSM nodes and edges saved successfully!")

ox.save_graphml(bicycle_graph, osm_graph_fp)
ox.save_graphml(bicycle_graph_simplified, osm_graph_simplified_fp)
print("OSM networks saved successfully!")

data_load_dict = {}

# Save time for when OSM data was loaded
current_time = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
data_load_dict["data_retrieved_at"] = current_time

with open(osm_meta_fp, "w") as outfile:
    json.dump(data_load_dict, outfile)

In [ ]:
# Export grid

grid.to_file(osm_grid_fp)
print("OSM grid saved successfully!")